In [1]:
import pandas as pd
import numpy as np

#reading cleaned data
df = pd.read_csv("cleaned_data.csv")
result = df['result']
#hot coding the catergorical data to 1 or 0
df = pd.get_dummies(df, columns=["cap-shape","cap-color","odor","gill-attachment","gill-spacing","gill-size","gill-color","stalk-shape","stalk-root","stalk-surface-above-ring","stalk-surface-below-ring","stalk-color-above-ring","stalk-color-below-ring","veil-type","veil-color","ring-type","spore-print-color","population","habitat"])
df = df.drop(columns=["result", "cap-shape.1", "veil-type_p"])


In [9]:
columns = list(df.columns.values)
correlations = {}
for column in columns:
    corr = df[column].corr(result)
    correlations.update({column:corr})
    #printing out semi-strong correlations 
    if(corr>.5 or corr<-.5):
        print(column + " ")
        print(corr)
        
#printing out all correlations
for key, value in correlations.items():
    print("%s : %f" % (key, value))


bruises 
0.5015303774075581
odor_f 
-0.6238419598140365
odor_n 
0.785556622236695
gill-size_b 
0.5400243574330168
gill-size_n 
-0.5400243574330168
gill-color_b 
-0.5388081615534451
stalk-surface-above-ring_k 
-0.5876582576306938
stalk-surface-below-ring_k 
-0.5735240117095741
ring-type_p 
0.5404689127747087
bruises : 0.501530
ring-number : 0.214366
cap-shape_b : 0.182567
cap-shape_c : -0.023007
cap-shape_f : -0.018526
cap-shape_k : -0.163565
cap-shape_s : 0.060664
cap-shape_x : 0.026886
cap-color_b : -0.067544
cap-color_c : 0.030910
cap-color_e : -0.097112
cap-color_g : 0.046456
cap-color_n : 0.044360
cap-color_p : -0.034702
cap-color_r : 0.042854
cap-color_u : 0.042854
cap-color_w : 0.133683
cap-color_y : -0.113014
odor_a : 0.219529
odor_c : -0.161278
odor_f : -0.623842
odor_l : 0.219529
odor_m : -0.069159
odor_n : 0.785557
odor_p : -0.186984
odor_s : -0.286360
odor_y : -0.286360
gill-attachment_a : 0.129200
gill-attachment_f : -0.129200
gill-spacing_c : -0.348387
gill-spacing_w : 0.3

In [3]:
import sklearn.linear_model
import sklearn.metrics 

#divide data half for training and half for testing
linear_data = df
size = int(linear_data.shape[0]/2)
train_features = linear_data[:4062]
train_result = result[:4062]
test_features = linear_data[4062:]
test_result = result[4062:]

#perform linear regression test
regr = sklearn.linear_model.LinearRegression()
regr.fit(train_features, train_result)
yts_hat = regr.predict(test_features)
coef_dict = {}
for coef, feat in zip(regr.coef_,columns):
    coef_dict[feat] = coef
    if(coef >=.1 or coef <=-.1):
        print(feat + " ")
        print(coef)
def convert_result(yhat):
    enumerated = []
    for num in yhat:
        ##.5 is chosen simply because its the average of 1 and 0 
        if(num >=.5):
            enumerated.append(1)
        else:
            enumerated.append(0)
    return enumerated

#convert result to 1 and 0 
enumerated = convert_result(yts_hat)

#RSS = np.mean((enumerated-test_result)**2)/(np.std(test_result)**2)

MSE = sklearn.metrics.mean_squared_error(enumerated, test_result)
accuracy = sklearn.metrics.accuracy_score(enumerated, test_result)
print("The accuracy is %f" % accuracy)

odor_a 
0.583328462289972
odor_f 
-0.15537398344206818
odor_l 
0.5833284622899733
odor_n 
0.6319861162598996
odor_p 
-0.372549106739741
odor_y 
-0.1542005904977743
gill-color_b 
-0.15254390475359864
gill-color_e 
0.13841582099852975
stalk-shape_e 
-0.21485918217055106
stalk-shape_t 
0.2146719299239735
stalk-root_b 
-0.3087130769265406
stalk-root_c 
0.1694756891379119
stalk-surface-below-ring_k 
0.19833152290223333
stalk-color-above-ring_e 
0.13563080186677193
ring-type_e 
-0.28118503037710146
ring-type_l 
0.19833152290223333
spore-print-color_h 
-0.15702325376002332
population_c 
0.13563080186677193
habitat_w 
0.1366456013431906
The accuracy is 0.959626


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [8]:
#k-fold test
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

kfold_data = df
#12 fold in this case
kf = KFold(n_splits=12, shuffle = True)
processed_data = preprocessing.scale(kfold_data)
accuracy = []
current_iteration = 1
for tr_index, ts_index in kf.split(processed_data):
    train_features, test_features = processed_data[tr_index], processed_data[ts_index]
    train_result, test_result = result[tr_index], result[ts_index]
    kfold_regr = sklearn.linear_model.LinearRegression()
    kfold_regr.fit(train_features, train_result)
    yts_hat = kfold_regr.predict(test_features)
    enumerated = convert_result(yts_hat)
    accuracy = sklearn.metrics.accuracy_score(enumerated, test_result) * 100
    print("Current iteration is # %d, the accuracy is %f" % (current_iteration, accuracy))
    current_iteration +=1
    #RSS = np.mean((enumerated-test_result)**2)/(np.std(test_result)**2)
    #print(sklearn.metrics.accuracy_score(enumerated, test_result))
print("The average accuracy is %f" % np.mean(accuracy))

Current iteration is # 1, the accuracy is 100.000000
Current iteration is # 2, the accuracy is 100.000000
Current iteration is # 3, the accuracy is 100.000000
Current iteration is # 4, the accuracy is 100.000000
Current iteration is # 5, the accuracy is 100.000000
Current iteration is # 6, the accuracy is 100.000000
Current iteration is # 7, the accuracy is 100.000000
Current iteration is # 8, the accuracy is 100.000000
Current iteration is # 9, the accuracy is 100.000000
Current iteration is # 10, the accuracy is 100.000000
Current iteration is # 11, the accuracy is 100.000000
Current iteration is # 12, the accuracy is 100.000000
The average accuracy is 100.000000


In [5]:
#using perceptron for binary classification 
from sklearn.linear_model import Perceptron

clf = Perceptron()
data = df
size = int(data.shape[0]/2)
train_features = data[:4062]
train_result = result[:4062]
test_features = data[4062:]
test_result = result[4062:]
clf.fit(train_features, train_result)
yts_hat = clf.predict(test_features)
clf.score(test_features, test_result)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.9276218611521418

In [6]:
#performing kfold for perceptron classifier 
kfold_data = df
#12 fold in this case
kf = KFold(n_splits=12, shuffle = True)
processed_data = preprocessing.scale(kfold_data)
accuracy = []
current_iteration = 1
for tr_index, ts_index in kf.split(processed_data):
    train_features, test_features = processed_data[tr_index], processed_data[ts_index]
    train_result, test_result = result[tr_index], result[ts_index]
    kfold_clf = Perceptron()
    kfold_clf.fit(train_features, train_result)
    yts_hat = kfold_clf.predict(test_features)
    enumerated = convert_result(yts_hat)
    accuracy = clf.score(test_features, test_result)
    print("Current iteration is # %d, the accuracy is %f" % (current_iteration, accuracy))
    current_iteration +=1
    #RSS = np.mean((enumerated-test_result)**2)/(np.std(test_result)**2)
    #print(sklearn.metrics.accuracy_score(enumerated, test_result))
print("The average accuracy is %f" % np.mean(accuracy))

Current iteration is # 1, the accuracy is 0.967504
Current iteration is # 2, the accuracy is 0.943870
Current iteration is # 3, the accuracy is 0.955687


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Current iteration is # 4, the accuracy is 0.958641
Current iteration is # 5, the accuracy is 0.964549
Current iteration is # 6, the accuracy is 0.948301
Current iteration is # 7, the accuracy is 0.955687
Current iteration is # 8, the accuracy is 0.957164
Current iteration is # 9, the accuracy is 0.966027
Current iteration is # 10, the accuracy is 0.942393
Current iteration is # 11, the accuracy is 0.964549
Current iteration is # 12, the accuracy is 0.958641
The average accuracy is 0.958641
